In [ ]:
# TODO UPDATE THIS OLD MESSY CODE WITH NEW PANEL

In [55]:
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import scipy.stats

In [92]:
# LOAD IN DATA
with open('../3-data/clean/asset_universe_dates_and_lists.pkl', 'rb') as handle:
    asset_universe_dict = pickle.load(handle) 
df = pd.read_csv('../3-data/panel_factor_analysis.csv')

# CLEAN UP CSV FILE GIVEN IT ISNT PKL
df = df.drop('Unnamed: 0', axis=1)
df['date'] = pd.to_datetime(df.date)

# LOOP OVER TOKEN UNIVERSE TO SUBSET DF TO VALID ROWS
included_df = pd.DataFrame()

# Loop over asset_universe_dict len with i
for i in range(len(asset_universe_dict)):
    # Extract this quarter and its included assets
    date = list(asset_universe_dict.keys())[i]
    assets = asset_universe_dict[date]

    # Form start and end date for this window
    start_date = datetime.strptime(date, '%Y-%m-%d')
    
    end_date   = datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=3)
    
    # Extract asset dates
    temp_df = df[(df.asset.isin(assets)) & (df.date >= start_date) & (df.date < end_date)][['date', 'asset']]
    included_df = pd.concat((included_df, temp_df.reset_index(drop=True)))

# Keep just those included rows
assert(included_df.shape[0] == included_df[~included_df.duplicated()].shape[0]) # ensure included df is unique
df = df.merge(included_df,
              on=['date', 'asset'],
              how='inner',
              validate='one_to_one')


In [ ]:

# ADD YEAR COLUMN AND BUILD LIST OF RHS
df['year'] = df['date'].dt.year

rhs_cols = df.columns.values.tolist()
to_remove = ['date', 'asset', 'mcap_t', 'week_idx', 'r_tplus7', 'year']
for var in to_remove:
    rhs_cols.remove(var)   
   

Extra argument geoavg (0: simple time series average, 1: geometric average)\
Extra argument flip_tertiles that will optionally flip tertiles or not.


In [2]:
def univariateFactorAnalaysis(df, factor, geoavg=True, flip_tertiles=False):
    
    # create result df
    res_col = ['1', '2', '3', '3-1' , 'factor', 'sort']
    res_row = ['2016', '2017', '2018', '2019', '2020', 'all', 'all_tstat']
    res_df = pd.DataFrame(np.nan, index=res_row, columns=res_col)
    res_df = res_df.assign(factor=factor)

    
    # drop cols from df
    temp_df = df[['date', 'asset', 'r_tplus7', 'mcap_t', 'year', factor]].copy()
    
    
    # Form the tertiles based on given factor
    if(flip_tertiles):
        temp_df = temp_df.sort_values(by=['date', factor], ascending=False)
    else:
        temp_df = temp_df.sort_values(by=['date', factor])
    temp_df['ranking'] = temp_df.groupby(['date']).cumcount()+1
    temp_df['counts'] = 1
    temp_df['assets_per_week'] = temp_df.groupby('date').counts.transform(lambda x: x.sum())
    temp_df['ranking'] = temp_df.ranking / temp_df.assets_per_week
    temp_df.loc[temp_df.ranking <= 1/3, 'tertile'] = 1
    temp_df.loc[(temp_df.ranking > 1/3) & (temp_df.ranking <= 2/3), 'tertile'] = 2
    temp_df.loc[(temp_df.ranking > 2/3) & (temp_df.ranking <= 1), 'tertile'] = 3

    
    # Calculate value-weighted average returns for each tertile
    temp_df['mcap_sum']    = temp_df.groupby(['date', 'tertile'])['mcap_t'].transform('sum')
    temp_df['weight']      = temp_df.mcap_t / temp_df.mcap_sum
    temp_df['tertile_r_tplus7'] = temp_df.weight * temp_df.r_tplus7
    temp_df['tertile_r_tplus7'] = temp_df.groupby(['date', 'tertile'])['tertile_r_tplus7'].transform('sum')
    temp_df.drop_duplicates(['date', 'tertile'], inplace=True)
    
    
    # Determine if factor is pos or neg corr with returns
    long_short_diff_overall = (np.mean(temp_df[temp_df.tertile==3].tertile_r_tplus7.values) 
                               - np.mean(temp_df[temp_df.tertile==1].tertile_r_tplus7.values))
    if long_short_diff_overall > 0:
        res_df['sort'] = '1_low_to_3_high'
    else:
        temp_df.loc[temp_df.tertile == 3, 'tertile'] = 4 # change to something else first
        temp_df.loc[temp_df.tertile == 1, 'tertile'] = 3
        temp_df.loc[temp_df.tertile == 4, 'tertile'] = 1
        res_df['sort'] = '1_high_to_3_low'

    
    # Take the geometric average (and t_stat) of tertiles 1-3
    # by year
    for year in [2016, 2017, 2018, 2019, 2020]: 
        year_df = temp_df[temp_df['year'] == year]
        for t in [1, 2, 3]:
            tertile_df = year_df[year_df['tertile'] == t]
            tertile_simple_returns = tertile_df.tertile_r_tplus7.values
            mean = 0
            if(geoavg): # geometric mean
                gmean = np.prod(tertile_simple_returns+1)**(1/len(tertile_simple_returns))-1
                mean = gmean
            else: # arithmetic mean
                amean = np.average(tertile_simple_returns+1)
                mean = amean
            res_df.iloc[int(year-2016),int(t-1)] = mean



            
    # and overall
    for t in [1.0, 2.0, 3.0]:
        tertile_df = temp_df[temp_df['tertile'] == t]
        tertile_simple_returns = tertile_df.tertile_r_tplus7.values
        # mean
        mean = 0
        if(geoavg): # geometric mean
            gmean = np.prod(tertile_simple_returns+1)**(1/len(tertile_simple_returns))-1
            mean = gmean
        else: # arithmetic mean
            amean = np.average(tertile_simple_returns+1)
            mean = amean
        res_df.iloc[int(year-2016),int(t-1)] = mean
        # t-stat
        t_stat = (np.mean(tertile_simple_returns) / 
                   np.std(tertile_simple_returns)*np.sqrt(len(tertile_simple_returns)))
        res_df.iloc[6,int(t-1)] = t_stat

        
    # Form the week by week 3-1 portfolio
    t3_df = temp_df[temp_df['tertile'] == 3]
    t1_df = temp_df[temp_df['tertile'] == 1]
    t3_df.rename(columns = {'tertile_r_tplus7':'t3_r'}, inplace = True)
    t1_df.rename(columns = {'tertile_r_tplus7':'t1_r'}, inplace = True)
    t3_1_df = t3_df.merge(t1_df,on='date',how='inner',validate='one_to_one')
    t3_1_df['tertile_r_3_1_t'] = t3_1_df.t3_r - t3_1_df.t1_r 
    t3_1_df = t3_1_df[['date','tertile_r_3_1_t']]
    t3_1_df['tertile'] = '3-1'
    t3_1_df['year'] = t3_1_df['date'].dt.year
    
    
    # fix returns below -1 to just blow up the strategy with a strict -1
    t3_1_df.loc[t3_1_df.tertile_r_3_1_t < -1, 'tertile_r_3_1_t'] = -1 
    

    # Take the average (and t_stat) of 3-1 portfolio
    # by year
    for year in [2016, 2017, 2018, 2019, 2020]: 
        year_df = t3_1_df[t3_1_df['year'] == year]
        simple_returns = year_df.tertile_r_3_1_t.values
        mean = 0
        if(geoavg): # geometric mean
            gmean = np.prod(simple_returns+1)**(1/len(simple_returns))-1
            mean = gmean
        else: # arithmetic mean
            amean = np.average(simple_returns+1)
            mean = amean
        res_df.iloc[int(year-2016),3] = mean
        
    # overall 
    simple_returns = t3_1_df.tertile_r_3_1_t.values
    mean = 0
    if(geoavg): # geometric mean
        gmean = np.prod(simple_returns+1)**(1/len(simple_returns))-1
        mean = gmean
    else: # arithmetic mean
        amean = np.average(simple_returns+1)
        mean = amean
    res_df.iloc[5,3] = mean


    # t-stat
    t_stats = np.mean(simple_returns)/np.std(simple_returns)*np.sqrt(len(simple_returns))
    res_df.iloc[6,3] = t_stats



    return res_df

In [ ]:
final_df = pd.DataFrame()
for factor in rhs_cols:
    output_df = univariateFactorAnalaysis(df, factor)
    final_df = pd.concat((final_df, output_df))


In [ ]:

fp = '../4-output/univariate_factor_analysis.xlsx'

with pd.ExcelWriter(fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
    final_df.to_excel(writer,sheet_name='raw_univariate')
